<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/evaluation/probe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 5.5 MB 14.8 MB/s 
     |████████████████████████████████| 7.6 MB 38.1 MB/s 
     |████████████████████████████████| 182 kB 38.9 MB/s 


In [ ]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 10.3 MB/s 


In [ ]:
import os
import numpy as np
import pandas as pd

import torch

from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration

import pprint
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option("display.max_colwidth", None)

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


### Black

In [ ]:
!pip install black[jupyter]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 12.8 MB/s 
     |████████████████████████████████| 96 kB 7.1 MB/s 
     |████████████████████████████████| 843 kB 61.0 MB/s 
     |████████████████████████████████| 1.6 MB 60.8 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [ ]:
!black "/content/drive/MyDrive/Colab Notebooks/probe.ipynb"

reformatted /content/drive/MyDrive/Colab Notebooks/probe.ipynb

All done! ✨ 🍰 ✨
1 file reformatted.


### Probe Class

A probe instance can be used to generate predictions from any of our models.

It loads (and caches) appropriate tokenizers and models as needed.

The only method you need to use is the predict method.

In [ ]:
class Probe:
    """
    A class used to represent an Awesome Puppies Question Generation Probe
    ...

    Attributes
    ----------
    models : dict
        a dictionary of cached models
    tokenizers : dict
        a dictionary of cached tokenizers

    Methods
    -------
    predict(context, answer, base_model='bart', training_dataset='amalgam',
            num_beams=4, early_stopping=True, no_repeat_ngram_size=3,
            maximum_input_length=1024, maximum_target_length=50)

        returns a prediction string

    retrieve_tokenizer (base_model='bart')

        returns and caches a tokenizer

    retrieve_model(base_model='bart', training_dataset='amalgam')

        returns and caches a model
    """

    project_root = "/content/drive/MyDrive/w266 NLP Final Project/"
    #    project_root = "/home/localadmin/Documents/w266_NLP_Final_Project/"
    model_root = project_root + "Models/"

    def __init__(self):
        self.models = {}
        self.tokenizers = {}

    def predict(
        self,
        context,
        answer,
        base_model="bart",
        training_dataset="amalgam",
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3,
        maximum_input_length=1024,
        maximum_target_length=50,
    ):

        tokenizer = self.retrieve_tokenizer(base_model)
        model = self.retrieve_model(base_model, training_dataset)

        if base_model == "bart":
            prompt_string = f"{answer} </s> {context}"
        elif base_model == "T5":
            prompt_string = f"generate question: answer: {answer} context: {context}"
        else:
            raise ValueError("invalid base model")

        inputs = tokenizer(
            prompt_string,
            return_tensors="pt",
            max_length=maximum_input_length,
            truncation=True,
            padding=True,
        )
        output_ids = model.generate(
            inputs["input_ids"].cuda(),
            max_length=maximum_target_length,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=early_stopping,
        )
        prediction = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        return prediction

    def retrieve_tokenizer(self, base_model="T5"):
        tokenizer = self.tokenizers.get(base_model)
        if tokenizer:
            return tokenizer

        if base_model == "bart":
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
        elif base_model == "T5":
            tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
        else:
            raise ValueError("invalid base model")

        self.tokenizers[base_model] = tokenizer
        return tokenizer

    def retrieve_model(self, base_model="bart", training_dataset="amalgam"):
        model_tuple = (base_model, training_dataset)
        model = self.models.get(model_tuple)
        if model:
            return model

        model_dir = f"{self.model_root}{base_model}_base_pt_long.{training_dataset}"

        if base_model == "bart":
            print(f"Loading: {model_dir}")
            model = BartForConditionalGeneration.from_pretrained(model_dir)
        elif base_model == "T5":
            print(f"Loading: {model_dir}")
            model = T5ForConditionalGeneration.from_pretrained(model_dir)
        else:
            raise ValueError("invalid base model")

        model.to(torch.device("cuda:0"))
        self.models[model_tuple] = model
        return model

In [ ]:
def panel(
    probe,
    context,
    answers,
    base_models,
    training_datasets,
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=3,
    maximum_input_length=1024,
    maximum_target_length=50,
):
    result = [
        {
            "Base Model": base_model,
            "Training Dataset": training_dataset,
            "Answer": answer,
            "Prediction": "".join(
                probe.predict(
                    context,
                    answer,
                    base_model=base_model,
                    training_dataset=training_dataset,
                    num_beams=num_beams,
                    no_repeat_ngram_size=no_repeat_ngram_size,
                    early_stopping=early_stopping,
                    maximum_input_length=maximum_input_length,
                    maximum_target_length=maximum_target_length,
                )
            ),
        }
        for answer in answers
        for base_model in base_models
        for training_dataset in training_datasets
    ]
    replacements = {
        "bart": "BART",
        "amalgam": "Shuffled Blended",
        "nq": "NQ",
        "quac": "QuAC",
        "squad": "SQuAD",
        "triviaqa": "TriviaQA",
    }
    return pd.DataFrame(result).replace(replacements)

### Play Time

In [ ]:
help(Probe)

Help on class Probe in module __main__:

class Probe(builtins.object)
 |  A class used to represent an Awesome Puppies Question Generation Probe
 |  ...
 |  
 |  Attributes
 |  ----------
 |  models : dict
 |      a dictionary of cached models
 |  tokenizers : dict
 |      a dictionary of cached tokenizers
 |  
 |  Methods
 |  -------
 |  predict(context, answer, base_model='bart', training_dataset='amalgam',
 |          num_beams=4, early_stopping=True, no_repeat_ngram_size=3,
 |          maximum_input_length=1024, maximum_target_length=50)
 |  
 |      returns a prediction string
 |  
 |  retrieve_tokenizer (base_model='bart')
 |  
 |      returns and caches a tokenizer
 |  
 |  retrieve_model(base_model='bart', training_dataset='amalgam')
 |  
 |      returns and caches a model
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  predict(self, context, answer, base_model='bart', training_dataset='amalg

In [ ]:
p = Probe()

In [ ]:
mary_had_a_little_lamb = "Mary had a little lamb, its fleece was white as snow. Everywhere the child went, the little lamb was sure to go."

In [ ]:
panel(
    p,
    mary_had_a_little_lamb,
    ["Mary", "little", "white"],
    ["T5", "bart"],
    ["amalgam", "squad"],
)

Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/T5_base_pt_long.amalgam
Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/T5_base_pt_long.squad
Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_long.amalgam
Loading: /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_long.squad


,Base Model,Training Dataset,Answer,Prediction
0,T5,Shuffled Blended,Mary,who had a little lamb that was white as snow
1,T5,SQuAD,Mary,Who had a little lamb?
2,BART,Shuffled Blended,Mary,who had a little lamb in the bible
3,BART,SQuAD,Mary,Who had a little lamb?
4,T5,Shuffled Blended,little,what was the size of mary's lamb
5,T5,SQuAD,little,What size was Mary's lamb?
6,BART,Shuffled Blended,little,how many lamb did mary have in the bible
7,BART,SQuAD,little,How big was Mary's lamb?
8,T5,Shuffled Blended,white,what color was the fleece of mary's lamb
9,T5,SQuAD,white,What color was Mary's lamb's fleece?


In [ ]:
fossie_had_a_little_lamb = "Fossie had a little lamb, its fleece was white as snow. Everywhere the puppy went, the little lamb was sure to go."

In [ ]:
df = panel(
    p,
    fossie_had_a_little_lamb,
    ["Fossie", "little", "white"],
    ["T5", "bart"],
    ["amalgam", "squad"],
)
df

,Base Model,Training Dataset,Answer,Prediction
0,T5,Shuffled Blended,Fossie,who had a little lamb in snow
1,T5,SQuAD,Fossie,Who had a little lamb?
2,BART,Shuffled Blended,Fossie,what was the name of the little lamb
3,BART,SQuAD,Fossie,Who had a little lamb?
4,T5,Shuffled Blended,little,what was the size of fossie's lamb
5,T5,SQuAD,little,What was the size of Fossie's lamb?
6,BART,Shuffled Blended,little,how many lamb does a Fossie have
7,BART,SQuAD,little,How big was Fossie's lamb?
8,T5,Shuffled Blended,white,what color was the fleece of fossie's lamb
9,T5,SQuAD,white,What color was Fossie's lamb's fleece?


In [ ]:
df.to_latex(index=False)

"\\begin{tabular}{llll}\n\\toprule\nBase Model & Training Dataset & Answer &                                 Prediction \\\\\n\\midrule\n        T5 & Shuffled Blended & Fossie &              who had a little lamb in snow \\\\\n        T5 &            SQuAD & Fossie &                     Who had a little lamb? \\\\\n      BART & Shuffled Blended & Fossie &       what was the name of the little lamb \\\\\n      BART &            SQuAD & Fossie &                     Who had a little lamb? \\\\\n        T5 & Shuffled Blended & little &         what was the size of fossie's lamb \\\\\n        T5 &            SQuAD & little &        What was the size of Fossie's lamb? \\\\\n      BART & Shuffled Blended & little &           how many lamb does a Fossie have \\\\\n      BART &            SQuAD & little &                 How big was Fossie's lamb? \\\\\n        T5 & Shuffled Blended &  white & what color was the fleece of fossie's lamb \\\\\n        T5 &            SQuAD &  white &     What col

In [ ]:
irving_had_a_little_lamb = "Irving had a little lamb, its fleece was white as snow. Everywhere the child went, the little lamb was sure to go."

In [ ]:
df = panel(
    p,
    irving_had_a_little_lamb,
    ["Irving", "little", "white"],
    ["T5", "bart"],
    ["amalgam", "squad"],
)
df

,Base Model,Training Dataset,Answer,Prediction
0,T5,Shuffled Blended,Irving,who had a little lamb that was white as snow
1,T5,SQuAD,Irving,Who had a little lamb?
2,BART,Shuffled Blended,Irving,who was the father of the little lamb
3,BART,SQuAD,Irving,Who had a little lamb?
4,T5,Shuffled Blended,little,What was Irving's lamb's size?
5,T5,SQuAD,little,What was Irving's lamb's size?
6,BART,Shuffled Blended,little,how many lamb did roberts have in the bible
7,BART,SQuAD,little,How big was Irving's lamb?
8,T5,Shuffled Blended,white,What color was Irving's fleece?
9,T5,SQuAD,white,What color was Irving's lamb's fleece?


In [ ]:
df.to_latex(index=False)

"\\begin{tabular}{llll}\n\\toprule\nBase Model & Training Dataset & Answer &                                   Prediction \\\\\n\\midrule\n        T5 & Shuffled Blended & Irving & who had a little lamb that was white as snow \\\\\n        T5 &            SQuAD & Irving &                       Who had a little lamb? \\\\\n      BART & Shuffled Blended & Irving &        who was the father of the little lamb \\\\\n      BART &            SQuAD & Irving &                       Who had a little lamb? \\\\\n        T5 & Shuffled Blended & little &               What was Irving's lamb's size? \\\\\n        T5 &            SQuAD & little &               What was Irving's lamb's size? \\\\\n      BART & Shuffled Blended & little &  how many lamb did roberts have in the bible \\\\\n      BART &            SQuAD & little &                   How big was Irving's lamb? \\\\\n        T5 & Shuffled Blended &  white &              What color was Irving's fleece? \\\\\n        T5 &            SQuAD &  

In [ ]:
irving_had_a_labradoodle = "Irving had a labradoodle named Fossie, its fur was white as snow. Everywhere that child went, the little dog was sure to go."

In [ ]:
df = panel(
    p,
    irving_had_a_labradoodle,
    ["Irving", "Fossie", "little", "white", "labradoodle"],
    ["T5", "bart"],
    ["amalgam", "squad"],
)
df

,Base Model,Training Dataset,Answer,Prediction
0,T5,Shuffled Blended,Irving,who had a labradoodle named fossie
1,T5,SQuAD,Irving,Who had a labradoodle named Fossie?
2,BART,Shuffled Blended,Irving,who had a dog called Fossie
3,BART,SQuAD,Irving,Who had a dog named Fossie?
4,T5,Shuffled Blended,Fossie,what was the name of irving's dog
5,T5,SQuAD,Fossie,What was Irving's labradoodle?
6,BART,Shuffled Blended,Fossie,what was the name of the dog in the movie
7,BART,SQuAD,Fossie,What was Irving's labradoodle named?
8,T5,Shuffled Blended,little,What was Fossie's size?
9,T5,SQuAD,little,What was Fossie's size?


In [ ]:
df.to_latex(index=False)

"\\begin{tabular}{llll}\n\\toprule\nBase Model & Training Dataset &      Answer &                                    Prediction \\\\\n\\midrule\n        T5 & Shuffled Blended &      Irving &            who had a labradoodle named fossie \\\\\n        T5 &            SQuAD &      Irving &           Who had a labradoodle named Fossie? \\\\\n      BART & Shuffled Blended &      Irving &                   who had a dog called Fossie \\\\\n      BART &            SQuAD &      Irving &                   Who had a dog named Fossie? \\\\\n        T5 & Shuffled Blended &      Fossie &             what was the name of irving's dog \\\\\n        T5 &            SQuAD &      Fossie &                What was Irving's labradoodle? \\\\\n      BART & Shuffled Blended &      Fossie &     what was the name of the dog in the movie \\\\\n      BART &            SQuAD &      Fossie &          What was Irving's labradoodle named? \\\\\n        T5 & Shuffled Blended &      little &                       Wha

In [ ]:
goldilocks = """Once upon a time there were three bears who lived in a house in the forest. 
There was a great big father bear, a middle-sized mother bear and a tiny baby bear.
One morning, their breakfast porridge was too hot to eat, so they decided to go for a walk in the forest. 
While they were out, a little girl called Goldilocks came through the trees and found their house. 
She knocked on the door and, as there was no answer, she pushed it open and went inside.
In front of her was a table with three chairs, one large chair, one middle-sized chair and one small chair. 
On the table were three bowls of porridge, one large bowl, one middle-sized bowl and one small bowl – and three spoons.
Goldilocks was hungry and the porridge looked good, so she sat in the great big chair, picked up the large spoon and tried some of the porridge from the big bowl. 
But the chair was very big and very hard, the spoon was heavy and the porridge too hot.
Goldilocks jumped off quickly and went over to the middle-sized chair. 
But this chair was far too soft, and when she tried the porridge from the middle-sized bowl it was too cold. 
So she went over to the little chair and picked up the smallest spoon and tried some of the porridge from the tiny bowl.
This time it was neither too hot nor too cold. 
It was just right and so delicious that she ate it all up. 
But she was too heavy for the little chair and it broke in pieces under her weight.
Next Goldilocks went upstairs, where she found three beds. 
There was a great big bed, a middle-sized bed and a tiny little bed. 
By now she was feeling rather tired. 
So she climbed into the big bed and lay down. 
The big bed was very hard and far too big. 
Then she tried the middle-sized bed, but that was far too soft. 
So she climbed into the tiny little bed. 
It was neither too hard nor too soft. 
In fact, it felt just right, all cosy and warm. 
And in no time at all Goldilocks fell fast asleep.
In a little while, the three bears came back from their walk in the forest. 
They saw at once that someone had pushed open the door of their house and Father Bear looked around. 
Then roared with a growly voice.
Mother Bear said in a quiet gentle voice.
"Somebody has been sitting in my chair.
Then Little Bear said in small squeaky baby voice.
"Somebody has been sitting in my chair and has broken it!"
Then Father Bear looked at his bowl of porridge and saw the spoon in it and he said in his great big growly voice,
"SOMEBODY HAS BEEN EATING MY PORRIDGE"
Then Mother Bear saw that her bowl had a spoon in it, and said in her quiet voice.
"Somebody has been eating my porridge Little Bear looked at his porridge bowl and said in his small squeaky baby voice,
"Somebody has been eating my porridge, and has eaten it all up:-
Then the three bears went upstairs, and Father Bear saw at once that his bed was untidy, and he said in his great big growly voice,
"SOMEBODY HAS BEEN SLEEPING IN MY BED!"
Mother Bear saw that her bed, too, had the bedclothes turned back, and she said in her quiet gentle voice,
"Somebody has been sleeping in my bed!"
Then Little Bear looked at his bed and said in his small squeaky baby voice,
"Somebody is sleeping in my bed!"
He squeaked so loudly that Goldilocks woke up with a start. She jumped out of bed, and away she ran, down the stairs and out into the forest. And the three bears never saw her again."""

In [ ]:
panel(
    p,
    goldilocks,
    ["the forest", "it broke", "it was too hot", "tiny"],
    ["T5", "bart"],
    ["amalgam", "squad"],
)

,Base Model,Training Dataset,Answer,Prediction
0,T5,Shuffled Blended,the forest,Where did the three bears live?
1,T5,SQuAD,the forest,Where did the three bears go for a walk?
2,BART,Shuffled Blended,the forest,Where did Goldilocks live?
3,BART,SQuAD,the forest,Where did the three bears live?
4,T5,Shuffled Blended,it broke,What happened to the little chair?
5,T5,SQuAD,it broke,What happened to the little chair?
6,BART,Shuffled Blended,it broke,What happened to the little chair?
7,BART,SQuAD,it broke,What happened to the little chair?
8,T5,Shuffled Blended,it was too hot,What was the problem with the porridge?
9,T5,SQuAD,it was too hot,What was the problem with the porridge?


In [ ]:
big_game = """BERKELEY - Jackson Sirmon scored on a 37-yard return following a wild play with two fumbles and the Cal football team rallied from a double-digit deficit in the fourth quarter to beat Stanford 27-20 in the 125th edition of the Big Game on Saturday.
"It's really neat... as a defensive player, you don't get a lot of opportunities to score touchdowns and you have to enjoy them when you get them," said Sirmon, who played in his first Big Game on Saturday. "It was very neat especially considering the timing of it all.""
The Golden Bears (4-7, 2-6 Pac-12) trailed 17-6 in the fourth quarter when Jack Plummer started the comeback with a 1-yard touchdown pass to Monroe Young.
Then things really went crazy on the ensuing possession for Stanford (3-8, 1-8).
Backup quarterback Ashton Daniels took a direct snap and ran to his left. He was stripped of the ball by Daniel Scott and Cal's Jeremiah Earby recovered. But Daniels then knocked the ball loose from Earby for a second fumble on the play only to have Sirmon scoop it up and run in to give Cal a 20-17 lead with 9:54 to play.
"It bounced right to me,'' Sirmon said." I was in the right place at the right time. All I did that play is I didn't mess it up. The ball came right to me and I ran with it.
Stanford had three more drives to rally. But the Cardinal punted twice and Scott came up with an interception with just more than two minutes to play, setting up Jaydn Ott's 1-yard run that sealed the win in front of a sellout crowd of 51,892.
"We know how important this game is to us as a team, our administration and our support staff," Travers Family Head Football Coach Justin Wilcox said. "Our fans and students were just incredible tonight. What an awesome environment." """

In [ ]:
pprint.pprint(big_game)

('BERKELEY - Jackson Sirmon scored on a 37-yard return following a wild play '
 'with two fumbles and the Cal football team rallied from a double-digit '
 'deficit in the fourth quarter to beat Stanford 27-20 in the 125th edition of '
 'the Big Game on Saturday.\n'
 '"It\'s really neat... as a defensive player, you don\'t get a lot of '
 'opportunities to score touchdowns and you have to enjoy them when you get '
 'them," said Sirmon, who played in his first Big Game on Saturday. "It was '
 'very neat especially considering the timing of it all.""\n'
 'The Golden Bears (4-7, 2-6 Pac-12) trailed 17-6 in the fourth quarter when '
 'Jack Plummer started the comeback with a 1-yard touchdown pass to Monroe '
 'Young.\n'
 'Then things really went crazy on the ensuing possession for Stanford (3-8, '
 '1-8).\n'
 'Backup quarterback Ashton Daniels took a direct snap and ran to his left. He '
 "was stripped of the ball by Daniel Scott and Cal's Jeremiah Earby recovered. "
 'But Daniels then knoc

In [ ]:
answers = [
    "Jakson Sirmon",
    "Stanford",
    "Jaydn Ott",
    "51,892",
    "17-6",
    "20-17",
    "27-20",
    "three-drives",
]
df = panel(p, big_game, answers, ["T5"], ["amalgam", "squad"])
df

,Base Model,Training Dataset,Answer,Prediction
0,T5,Shuffled Blended,Jakson Sirmon,Who scored on a 37-yard return in the Big Game?
1,T5,SQuAD,Jakson Sirmon,Who scored on a 37-yard return?
2,T5,Shuffled Blended,Stanford,Who did Cal beat in the Big Game?
3,T5,SQuAD,Stanford,Who did the Golden Bears beat in the 125th edition of the Big Game?
4,T5,Shuffled Blended,Jaydn Ott,Who scored a 1-yard run in the final quarter of the Big Game?
5,T5,SQuAD,Jaydn Ott,Who scored a 1-yard run in the fourth quarter?
6,T5,Shuffled Blended,"51,892",How many people attended the game?
7,T5,SQuAD,"51,892",How many people attended the game?
8,T5,Shuffled Blended,17-6,How many points did Cal trail in the fourth quarter?
9,T5,SQuAD,17-6,How many points did the Golden Bears trail in the fourth quarter?


In [ ]:
df.to_latex(index=False)

'\\begin{tabular}{llll}\n\\toprule\nBase Model & Training Dataset &        Answer &                                                          Prediction \\\\\n\\midrule\n        T5 & Shuffled Blended & Jakson Sirmon &                     Who scored on a 37-yard return in the Big Game? \\\\\n        T5 &            SQuAD & Jakson Sirmon &                                     Who scored on a 37-yard return? \\\\\n        T5 & Shuffled Blended &      Stanford &                                   Who did Cal beat in the Big Game? \\\\\n        T5 &            SQuAD &      Stanford & Who did the Golden Bears beat in the 125th edition of the Big Game? \\\\\n        T5 & Shuffled Blended &     Jaydn Ott &       Who scored a 1-yard run in the final quarter of the Big Game? \\\\\n        T5 &            SQuAD &     Jaydn Ott &                      Who scored a 1-yard run in the fourth quarter? \\\\\n        T5 & Shuffled Blended &        51,892 &                                  How many people at